# HMDA Cleaner

The purpose of this program is to clean HMDA data further data needs.

This program is designed to be used for future data HMDA publications.

## Import HMDA Data, Packages, and Check Dataset.

In [1]:
#This code does not need to be changed.
#Imports packages to be used by program
import pandas as pd
import numpy as np

#This code does not need to be changed.
#Quality of Life settings
#This will allow all columns to be displayed when reviewing the data.
pd.options.display.max_columns = None

In [2]:
'''
This code does not need to be changed unless column headers change.(Ex.'state_code' changes to 'state')
Program to call for future HDMA data.
Sample size is an optional argument if one wants to take a sample of the dataset enter the size of the desired sample.
Don't enter anything if full dataset is desired.
'''
def HMDA_cleaner(list_of_datasets, Name_of_Output_CSV, sample_size = 0):
    #load in dataset.
    #HMDA_dataset = pd.read_csv(HMDA_dataset_file_location)
    
    HMDA_dataset = pd.DataFrame()
    for i in list_of_datasets:
        i_dataset = pd.read_csv(i)
        HMDA_dataset = pd.concat([HMDA_dataset, i_dataset])
    
    
    '''
    Optional Sample Size Argument
    '''
    if sample_size > 0:
        df = HMDA_dataset.sample(n = sample_size)
    else:
        df = HMDA_dataset
    
    '''
    Clean Race data.
    '''
    #Groups racial categories and adds names.
    #The '0_' in the '0_White' indicates that it will be the ommited variable in a regression.
    df1 = df.replace(to_replace = {'applicant_race-1' : {
                                    (5):'0_White',
                                    (2,21,22,23,24,25,26,27):'Asian',
                                    (3):'Black',
                                    (1, 4,41,42,43,44):'Other',
                                    (6,7, np.nan):'Not Reported'}})
    #Remove 'Not Reported' rows.
    df2 = df1[df1['applicant_race-1'] != 'Not Reported']
    #Clean Ethnicity Data.
    df3 = df2.replace(to_replace = {'applicant_ethnicity-1' : {
                                (1,11,14,12,13):1,
                                  (2):0,
                                 (3,4,np.nan):'Not Reported',}})
    #Remove 'Not Reported' rows.
    df4 = df3[df3['applicant_ethnicity-1'] != 'Not Reported']
    
    #Create Race Column.
    df4 = df4.reset_index()
    race = []
    for row in df4.index:
        if df4['applicant_ethnicity-1'][row] == 1:
            race.append('Latinx')
        else:
            race.append(df4['applicant_race-1'][row])
    df4_copy = df4.copy()
    df4_copy['Race'] = race
    
    '''
    Clean Income 
    '''
    #Remove rows that had no income reported or an income less than 0.
    def income_reported_to_categorical(table, column):
        values = table[column].values
        new_table = table.copy()
        ir_array =  []
        for i in values:
            if i == np.nan:
                ir_array.append('No Report')
            elif i <= 0:
                ir_array.append('Zero or Negative')
            elif i > 0:
                ir_array.append('Reported')
            else:
                ir_array.append('No Report')
        new_table['Income Reported'] = ir_array
        return new_table
    #This removes all non reported and 0 or negative rows.
    df6 = income_reported_to_categorical(df4_copy,'income')
    df7 = df6[df6["Income Reported"].str.contains("Reported") == True]
    #Creates log Income column.
    df7_2 = df7.copy()
    df7_2['Log Income'] = np.log(df7['income'])
    
    '''
    Create approval and denial indicator column.
    This also removes withdrawn applications and other instances that aren't under the Approval or Denial categories.
    '''
    def action_taken_to_approval_indicator(table, column):
        values = table[column].values
        ir_array =  []
        for i in values:
            if i == 1:
                ir_array.append('Approved')
            elif i == 2:
                ir_array.append('Approved')
            elif i == 3:
                ir_array.append('Denied')
            else:
                ir_array.append('other')
        table2 = table.copy()
        table2['Approval Indicator'] = ir_array
        table3 = table2[table2["Approval Indicator"].str.contains("other") == False]
        table4 = pd.get_dummies(table3, columns = ['Approval Indicator'])
        return table4
    df8 = action_taken_to_approval_indicator(df7_2, 'action_taken')
   
    '''
    Clean applicant sex
    '''
    df9 = df8.replace(to_replace = {'applicant_sex' : {(1):'0_Male',(2):'Female',
                                                      (3,4,6):'Not Applicable'}})
    df10 = df9[df9['applicant_sex'] != 'Not Applicable']
    #df10 = pd.get_dummies(df9, columns = ['applicant_sex'])
    
    '''
    Clean Loan to Value Ratios
    '''
    #This removes all non-numeric rows.
    df11 = df10[pd.to_numeric(df10['loan_to_value_ratio'], errors = 'coerce').notnull()]
    
    '''
    Clean Debt to Income    
    '''
    #Creates a second DTI column and changes name for OLS.
    df11_01 = df11.copy()
    df11_01['DTI_Ratio'] = df11_01['debt_to_income_ratio']
    df11_02 = df11_01.replace(to_replace = {'DTI_Ratio': {'<20%':'0%-20%'}})
    #Drops no answer rows.
    df12 = df11_02.dropna(subset=['debt_to_income_ratio'])
    #Drops exempt rows.
    df13 = df12[df12["debt_to_income_ratio"].str.contains("Exempt") == False]
    #Rename Column
    df13_5 = df13.rename(columns = {'debt_to_income_ratio': 'DTI'})
    df13_4 =  pd.get_dummies(df13_5, columns = ['DTI'])
   
    
    '''
    Clean Loan Amount and create log(Loan Amount)
    '''
    #This drops all N/A rows.
    df14 = df13_4.dropna(subset=['loan_amount'])
    #This removes all non-numeric rows.
    df15 = df14[pd.to_numeric(df14['loan_amount'], errors = 'coerce').notnull()]
    #This removes all rows with a value less than or equal to 0.
    df16 = df15[~(df15['loan_amount'] <= 0)]
    #Creates log(Loan Amount)
    df16['Log Loan Amount'] = np.log(df16['loan_amount'])
    
    '''
    Clean Loan Type
    '''
    df17 = df16.replace(to_replace = {'loan_type' : {(1):'Conventional',
                                                    (2): 'FHA',(3): 'VA',
                                                    #USDA Rural Housing Service or Farm Service Agency.
                                                    (4): 'RHS or FSA'}})
    df18 = df17.rename(columns = {'loan_type': 'Loan_Type'})
    
    '''
    Filter Loan Purpose
    '''
    #This study is only interested in Home Purchase loans.
    #It is leaving out home improvement, refinancing, Cash-out refinanciing, Not applicable, and other purposes.
    df20 = df18[df18["loan_purpose"] == 1]
    
    '''
    Clean Preapproval
    '''
    df21 = df20.replace(to_replace = {'preapproval' : {(1): 'Preapproval Requested',
                                                       (2): '0 No Preapproval Request'}})
     
    '''
    Compile final Data Frame
    '''
    #Rename columns
    dfrename = df21.rename(columns = {'Approval Indicator_Approved' : 'Approved',
                                       'Approval Indicator_Denied' : 'Denied',
                                        'income': 'Income',
                                        'applicant_ethnicity-1': 'Ethnicity',
                                        'applicant_sex': 'Sex',
                                        'loan_to_value_ratio': 'LTV',
                                        'loan_amount': 'Loan_Amount',
                                        'denial_reason-1': 'Denial Reason',
                                        'state_code' : 'State',
                                        'county_code':'County_Code',
                                        'census_tract': 'Census_Tract',
                                        'activity_year': 'Year',
                                        'lei': 'Lender_LEI',
                                        'preapproval' : 'Preapproval',
                                        'Log Income':'Log_Income',
                                        'Log Loan Amount': 'Log_Loan_Amount',
                                        'occupancy_type':'Occupancy_Type'})
    #Pull columns
    Final_df = dfrename[['Year','Lender_LEI','State','County_Code','Census_Tract','Approved','Denied', 
                         'Race','Sex','Income','Log_Income', 
                         'Loan_Amount','Log_Loan_Amount','LTV','Loan_Type','DTI_Ratio','Preapproval','Occupancy_Type']]
    #Add index column.
    Final_df2 = Final_df.copy()
    Final_df2 = Final_df2.reset_index()
    
    '''
    Save the Dataframe to a csv
    '''
    Final_df2.to_csv(Name_of_Output_CSV, index=False)
    
    '''
    Outputs
    '''
    print('Data written to CSV sucessfully.')
    print(Final_df2.shape[0],'Rows,',Final_df2.shape[1],'Columns.')
    #return Final_df.head()
    return Final_df2.head()

In [ ]:
#Run HMDA_cleaner.
#Enter file location in the HMDA_cleaner(r'file_location') format.|
HMDA_cleaner([r'C:\Users\raywz\Coding\HMDA_IL_2021.csv.',r'C:\Users\raywz\Coding\HMDA_IL_2020.csv.',r'C:\Users\raywz\Coding\state_IL_2019_Raw.csv.'], 
             'HMDA Clean IL SAMPLE.csv', 1000000)

In [3]:
HMDA_cleaner([r'C:\Users\raywz\Coding\HMDA_IL_2021.csv.',r'C:\Users\raywz\Coding\HMDA_IL_2020.csv.',r'C:\Users\raywz\Coding\state_IL_2019_Raw.csv.'], 
             'HMDA Clean IL SAMPLE.csv')

C:\Users\raywz\AppData\Local\Temp/ipykernel_15132/360051114.py:13: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44,46,77,78,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  i_dataset = pd.read_csv(i)
C:\Users\raywz\AppData\Local\Temp/ipykernel_15132/360051114.py:13: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  i_dataset = pd.read_csv(i)
C:\Users\raywz\AppData\Local\Temp/ipykernel_15132/360051114.py:13: DtypeWarning: Columns (22,23,24,26,27,28,29,30,31,32,33,38,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  i_dataset = pd.read_csv(i)


Data written to CSV sucessfully.
455870 Rows, 19 Columns.


,index,Year,Lender_LEI,State,County_Code,Census_Tract,Approved,Denied,Race,Sex,Income,Log_Income,Loan_Amount,Log_Loan_Amount,LTV,Loan_Type,DTI_Ratio,Preapproval,Occupancy_Type
0,2,2021,54930034MNPILHP25H80,IL,17163.0,1.716350e+10,1,0,0_White,0_Male,65.0,4.174387,235000.0,12.367341,76.0,Conventional,30%-<36%,0 No Preapproval Request,1
1,8,2021,54930034MNPILHP25H80,IL,17037.0,1.703700e+10,1,0,0_White,0_Male,88.0,4.477337,65000.0,11.082143,75.0,Conventional,20%-<30%,0 No Preapproval Request,3
2,13,2021,54930034MNPILHP25H80,IL,17197.0,1.719788e+10,1,0,0_White,0_Male,151.0,5.017280,665000.0,13.407542,98.23,VA,41,0 No Preapproval Request,1
3,149,2021,FT6J43S06X6CLJ0R0B48,IL,17031.0,1.703134e+10,1,0,Asian,0_Male,76.0,4.330733,245000.0,12.409013,68.572,Conventional,30%-<36%,0 No Preapproval Request,1
4,158,2021,FT6J43S06X6CLJ0R0B48,IL,17097.0,1.709786e+10,0,1,Asian,0_Male,48.0,3.871201,165000.0,12.013701,80.0,Conventional,>60%,0 No Preapproval Request,1


### The CSV is now ready to be used for further descriptive statistics and the LPM.